In [2]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

# 1. DB 접속 정보
db_host = "127.0.0.1"
db_port = "3307"
db_user = "root"
db_pass = "goodboyseongmin12!"
db_name = "crm"

# 테이블 정보
user_table = "users"
cart_table = "carts"
cart_item_table = "cart_items"
product_table = "products"
ocr_table = "product_ocr_text"

db_url = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)

def process_abandoned_cart_majority():
    print(f"📡 [Case 2] 장바구니 대세 상품 리타겟팅 시작...")

    # --- [Step 1] 최신 타겟 & 템플릿 데이터 조회 ---
    query_target = "SELECT payload_json FROM handoffs WHERE stage = 'TARGET_AUDIENCE' ORDER BY created_at DESC LIMIT 1"
    query_template = "SELECT payload_json FROM handoffs WHERE stage = 'SELECTED_TEMPLATE' ORDER BY created_at DESC LIMIT 1"

    try:
        df_target = pd.read_sql(query_target, engine)
        df_template = pd.read_sql(query_template, engine)
    except Exception as e:
        print(f"❌ DB 접속 실패: {e}")
        return

    if df_target.empty or df_template.empty:
        print("⚠️ 처리할 데이터가 없습니다.")
        return

    target_data = json.loads(df_target.iloc[0]['payload_json'])
    template_data = json.loads(df_template.iloc[0]['payload_json'])
    
    user_ids = target_data.get('user_ids', [])
    template_body = template_data.get('body_with_slots', "")
    
    if not user_ids:
        print("⚠️ 타겟 유저 ID가 없습니다.")
        return

    # --- [Step 2] 이탈 유저(교집합) 및 담긴 상품들 조회 ---
    ids_tuple = tuple(user_ids)
    in_clause = f"('{user_ids[0]}')" if len(user_ids) == 1 else str(ids_tuple)

    # 1. 타겟 유저 중 ABANDONED 상태인 사람들의 '담긴 상품(prod_sn)'을 모두 가져옵니다.
    cart_query = f"""
        SELECT 
            c.user_id,
            u.customer_name,
            ci.prod_sn
        FROM {cart_table} c
        JOIN {user_table} u ON c.user_id = u.user_id
        JOIN {cart_item_table} ci ON c.cart_id = ci.cart_id
        WHERE c.status = 'ABANDONED'
          AND c.user_id IN {in_clause}
    """
    
    try:
        cart_df = pd.read_sql(cart_query, engine)
    except Exception as e:
        print(f"❌ 장바구니 조회 실패: {e}")
        return

    if cart_df.empty:
        print("⛔ 대상 유저 중 장바구니 이탈 내역이 없습니다.")
        return

    # --- [Step 3] 가장 많이 담긴 '대세 상품' 1개 선정 (Majority Vote) ---
    # prod_sn 컬럼의 최빈값(Mode)을 찾습니다.
    most_popular_sn = cart_df['prod_sn'].value_counts().idxmax()
    count = cart_df['prod_sn'].value_counts().max()
    
    print(f"✅ 필터링된 대상 유저 수: {cart_df['user_id'].nunique()}명")
    print(f"🏆 [장바구니 최다 등장 상품]: {most_popular_sn} (총 {count}회 담김)")

    # --- [Step 4] 선정된 상품의 상세 정보 조회 ---
    # detail_url -> cta
    # detail_slot -> product_detail
    product_query = f"""
        SELECT 
            p.product_name,
            p.detail_url,
            o.detail_slot
        FROM {product_table} p
        LEFT JOIN {ocr_table} o ON p.prod_sn = o.prod_sn
        WHERE p.prod_sn = '{most_popular_sn}'
    """
    
    try:
        product_df = pd.read_sql(product_query, engine)
    except Exception as e:
        print(f"❌ 상품 정보 조회 실패: {e}")
        return
        
    if product_df.empty:
        print("⛔ 선정된 상품의 상세 정보가 없습니다.")
        return

    # 상품 정보 추출 (Null 처리 포함)
    winning_product = product_df.iloc[0]
    p_name = winning_product['product_name']
    p_cta = winning_product['detail_url'] if winning_product['detail_url'] else ""
    p_detail = winning_product['detail_slot'] if winning_product['detail_slot'] else ""
    p_offer = ""  # 👈 요청하신 대로 빈 값 처리

    print(f"👉 [최종 선정 상품]: {p_name}")
    print("-" * 50)

    # --- [Step 5] 메시지 생성 (모든 이탈 유저에게 동일 상품 적용) ---
    final_results = []
    
    # 중복 제거된 유저 리스트 (한 사람이 여러 개 담았을 수 있으므로 유저 단위로 유니크하게 만듦)
    target_users = cart_df[['user_id', 'customer_name']].drop_duplicates()

    print("\n[메시지 발송 미리보기]")
    for _, row in target_users.iterrows():
        uid = row['user_id']
        name = row['customer_name']
        
        slot_values = {
            "customer_name": name,
            "product_name": p_name,    # 대세 상품명
            "offer": p_offer,          # 빈 값 ("")
            "cta": p_cta,              # detail_url
            "product_detail": p_detail # detail_slot
        }

        try:
            completed_message = template_body.format(**slot_values)
            final_results.append({"user_id": uid, "message": completed_message})
            print(f"[{uid}/{name}] {completed_message}")
        except KeyError as e:
            print(f"❌ 슬롯 에러 ({uid}): {e}")

if __name__ == "__main__":
    process_abandoned_cart_majority()

📡 [Case 2] 장바구니 대세 상품 리타겟팅 시작...
✅ 필터링된 대상 유저 수: 1명
🏆 [장바구니 최다 등장 상품]: 57026 (총 1회 담김)
👉 [최종 선정 상품]: 레티놀 시카 흔적 앰플 50ml
--------------------------------------------------

[메시지 발송 미리보기]
[u_037/User037] User037님, 장바구니에 있는 레티놀 시카 흔적 앰플 50ml를 잊지 마세요!  혜택이 준비되어 있어요. 지금 앱에서 확인해 보세요: https://www.amoremall.com/kr/ko/product/detail?onlineProdSn=57026
· 고객님, 이니스프리 레티늘 시카 액트를 통해 매끄럽고 촉촉한 피부결을 경험해보세요! 압도적인 판매량이 이 제품의 효과를 증명합니다!
